### Using a TRILEGAL file for a given healpix, produce maps with Mr - [Fe/H] priors 

Version with a subsample of the hipscat TRILEGAL catalog processed with LSDB

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from dask.distributed import Client
pd.set_option('display.max_columns', None)

In [2]:
import os
import shutil
from collections import defaultdict

def delete_files_and_directories(directory):
    # Dictionary to store the count of each file extension
    extension_counts = defaultdict(int)
    file_count = 0
    dir_count = 0
    
    # Walk through the directory
    for root, dirs, files in os.walk(directory, topdown=False):
        # Delete files and collect their extensions
        for file in files:
            file_path = os.path.join(root, file)
            file_extension = os.path.splitext(file)[1]  # Get file extension
            extension_counts[file_extension] += 1
            os.remove(file_path)
            file_count += 1
        
        # Delete directories
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            shutil.rmtree(dir_path)
            dir_count += 1
    
    # Print the results
    print(f'Total files deleted: {file_count}')
    print(f'Total directories deleted: {dir_count}')
    print('File extensions and their counts:')
    for ext, count in extension_counts.items():
        print(f'{ext}: {count}')
    
    return file_count, dir_count, dict(extension_counts)

In [4]:
# client.close()
# client = Client(n_workers=96, threads_per_worker=1, memory_limit="auto", dashboard_address=':41987')
client = Client(n_workers=96, threads_per_worker=2, memory_limit="128GB", dashboard_address=':41987')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41987/status,
Dashboard: http://127.0.0.1:41987/status,Workers: 96
Total threads: 192,Total memory: 11.18 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34659,Workers: 96
Dashboard: http://127.0.0.1:41987/status,Total threads: 192
Started: Just now,Total memory: 11.18 TiB
Comm: tcp://127.0.0.1:42551,Total threads: 2
Dashboard: http://127.0.0.1:45397/status,Memory: 119.21 GiB
Nanny: tcp://127.0.0.1:43281,


## LSDB

In [7]:
%%time
import lsdb
from lsdb.core.search import BoxSearch

# trilegal = lsdb.read_hats("/mnt/beegfs/scratch/data/TRILEGAL/TRILEGAL_hats/TRILEGAL_level_5_cluster") #, search_filter=BoxSearch(ra=(-53, 61), dec=(-30., 30.)))
trilegal = lsdb.read_hats("/mnt/beegfs/scratch/data/TRILEGAL/TRILEGAL_hats/TRILEGAL_level_5_cluster")

# trilegal.skymap(lambda df, a: np.log10(len(df)+1), order=9)

CPU times: user 486 ms, sys: 85.1 ms, total: 571 ms
Wall time: 550 ms


VOMBAT:

-12, 12
192, 1, 128GB

CPU times: user 29 s, sys: 12.3 s, total: 41.4 s
Wall time: 1min 58s
??????????????????
___


-30, 30
192, 1, 128GB

CPU times: user 2min 16s, sys: 15.2 s, total: 2min 31s
Wall time: 2min 26s
___

client = Client(n_workers=96, threads_per_worker=1, memory_limit="auto", dashboard_address=':41987')
CPU times: user 2min 19s, sys: 10.9 s, total: 2min 30s
Wall time: 2min 24s
___

client = Client(n_workers=96, threads_per_worker=1, memory_limit="128GB", dashboard_address=':41987')
CPU times: user 2min 20s, sys: 11.2 s, total: 2min 31s
Wall time: 2min 26s

___

client = Client(n_workers=96, threads_per_worker=2, memory_limit="128GB", dashboard_address=':41987')
CPU times: user 2min 20s, sys: 9.2 s, total: 2min 29s
Wall time: 2min 23s
___

client = Client(n_workers=48, threads_per_worker=1, memory_limit="128GB", dashboard_address=':41987')
CPU times: user 2min 20s, sys: 9.26 s, total: 2min 30s
Wall time: 2min 25s
___

client = Client(n_workers=48, threads_per_worker=2, memory_limit="128GB", dashboard_address=':41987')

CPU times: user 2min 17s, sys: 7.85 s, total: 2min 25s
Wall time: 2min 21s

___
___
___

CLUSTER:



___
___
___

Bottom line: CPU occupancy is rather  low, because the whole data set is not balanced. The large pixels take a lot of time to process and therefore multiprocessing does not scale as well. 


In [ ]:
%%time

from photod import priors

def prep(df, pix):
    subdir = f'{rootdir}/{str(pix.order)}'
    rootname = f'{subdir}/{str(pix.pixel)}'
    
    if not os.path.exists(subdir):
        os.makedirs(subdir)
        print(f'Directory created: {subdir}')
    else:
        print(f"Directory already exists: {subdir}")


    priors.dumpPriorMaps_testing(df, rootname, pix, verbose=False)
    return pd.DataFrame(data=[(pix, len(df))], columns=['HealPix', 'len'])

rootdir = '/mnt/beegfs/scratch/seanmcgu/photod/priors/TRILEGAL/'  # TODO: clean this up!

# Perform the deletion and gather statistics
delete_files_and_directories(rootdir)
trilegal.map_partitions(prep, include_pixel=True).compute()